# 형태소 분석 및 워드클라우드

install file : java_jdk
pip install graphviz
pip install konlpy
pip install wordcloud (따로 설치 하던 중 붉은 글씨로 Microsoft Visual C++ 14.0 is required. Get it with "Microsoft Visual C++ Build Tools": 라는 에러가 나오면 Visual C++ Build Tools 설치할 것)

In [ ]:
import scipy as sp  # SCIENCE 관련
import pandas as pd
import numpy as np

# 사이킷런 = 머신러닝 관련 헤더
from sklearn.feature_extraction.text import CountVectorizer 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

from wordcloud import WordCloud, STOPWORDS
from PIL import Image

# 그래프 - 자연어처리는 항상 그래프를 그려 단어 빈도 측정
%matplotlib inline
import os

# 데이터 시각화
import matplotlib.pyplot as plt
import seaborn as sns
import graphviz             
from sklearn.tree import export_graphviz

import nltk # 파이썬 자연어 처리, nltk = nature langauge tool kit

# 그래프에서 한글 폰트 깨지는 문제에 대한 대처(전역 글꼴 설정)
from matplotlib import font_manager, rc
font_name = font_manager.FontProperties(fname='c:/Windows/Fonts/malgun.ttf').get_name()
                
rc('font', family=font_name)

# 코드 실행시 발생하는 Warnings 없애주는 것
import warnings  
warnings.filterwarnings('ignore')

In [ ]:
# Konlpy : 한글 자연어 처리 라이브러리!!! 
from konlpy.tag import Kkma       ; kkma = Kkma() # kkma()라는 클래스 안에서 kkma라는 객체를 생성하여 사용하겠다는 의미
from konlpy.tag import Hannanum   ; hannanum = Hannanum()
from konlpy.tag import Okt        ; t = Okt()  # Okt()라는 클래스 안에서 t라는 객체를 생성하여 아래에서 계속 쓰겠다고 표현
from konlpy.tag import *
import pickle

In [ ]:
df_r = pd.read_excel("blog_내용 크롤링.xlsx") # 저장된 엑셀 열기
df_r

In [ ]:
df_r['content'] # 내가 원하는 워드를 contet로 쓰고 싶으면 de_r["content"]  or  de_r.content로 쓰면 됨지만 앞서 []안에서는 띄어쓰기가 가능

In [ ]:
content_list = df_r.content.values.tolist() # 지금 선정한 전체값들을 list로 변환

print(len(content_list))
content_list # 이작업을 한것은 list로 바꾼것을 나중에 하나의 문장으로 바꾸기 위해서

In [ ]:
# 리스트 중 str 타입이 아닌 요소들이 존재함을 확인
for i in content_list:
    if type(i) == float:
        i = str(i)

In [ ]:
# str 타입이 아닌 요소들이 있으면 전부 str 타입으로 바꿔라! (종종 문자열이 아닌것도 존재)
for i in range(len(content_list)):
    if type(content_list[i]) != str:  
        content_list[i] = str(content_list[i])

In [ ]:
content_list[2] 

In [ ]:
# 나눠져 있는 문장을 한개의 문장으로 합치는 작업

content_text = '' # 빈 문자열

for each_line in content_list[:83]: # 수집한 개수 만큼 숫자를 변경
    content_text = content_text + each_line + '\n' # for문을 통해서 빈 문자열에 계속 넣도록 함

In [ ]:
content_text # 하나의 문장에 가공이 끝난 모든 문장을 합치도록 함

## 여기까지 데이터 전처리 작업

In [ ]:
# knolpy 라는 라이브러리에 Okt 모듈( =t) 중에 
# morphs 라는 메소드를 사용 = 이것은 글들을 쪼개서 형태소 분석을 하는 것, 형태소란 언어에 있어서 "최소 의미 단위"를 말한다. 
# nonus 라는 메소드는 명사만을 가져온다.

tokens_ko = t.morphs(content_text) 
tokens_ko

In [ ]:
ko = nltk.Text(tokens_ko)   # nltk = 빈도 분석 매서드 사용을 위해, nltk.Text를 하면 토큰화(token)(텍스트분석의기본단위)가됩니다
print(len(ko.tokens))          # 토큰 전체 갯수
print(len(set(ko.tokens)))     # 토큰 unique 갯수

In [ ]:
ko.vocab().most_common(100)    # 상위에서 가장 많이 나온 단어(토큰) 100개

In [ ]:
# 불용어 : 인터넷 검색 시 검색 용어로 사용하지 않는 단어. 관사, 전치사, 조사, 접속사 등 검색 색인 단어로 의미가 없는 단어
# stop_words list에 앞서 정제한 단어 중 필요 없는 단어를 추가 한다
stop_words = [')','?','1','"(', '_', ')/','\n','.',',', '<','!','(','(', '??','..', '4', '|', '>', '?(', '"…', '#', '&', '・', "']",'.',' ','/',"'",'’','”','“','·', '[','!','\n','·','‘','"','\n ',']',':','…',')','(','-', 'nan','가','요','답변','...','을','수','에','질문','제','를','이','도',
                      '좋','1','는','로','으로','2','것','은','다',',','니다','대','들',
                      '2017','들','데','..','의','때','겠','고','게','네요','한','일','할',
                      '10','?','하는','06','주','려고','인데','거','좀','는데','~','ㅎㅎ',
                      '하나','이상','20','뭐','까','있는','잘','습니다','다면','했','주려',
                      '지','있','못','후','중','줄','6','과','어떤','기본','!!',
                      '단어','라고','중요한','합','가요','....','보이','네','무지','했습니다',
              '이다','대해','에게','입니다','있다','사람','대한','3','합니다','및','장','에서','하고','검','한다','만',
             '적', '성', '삼', '등', '전', '인', '그', '했다', '와', '위', '해', '권', '된', '서', '말', '분']

# 리스트 컴프리헨션 사용 (리스트 안에 for문, if 문 등을 사용)
tokens_ko = [each_word for each_word in tokens_ko
             if each_word not in stop_words] # 불용어를 제거한 단어들만 선택

In [ ]:
ko = nltk.Text(tokens_ko) # stop_words만 빼고 새롭게 조사
ko.vocab().most_common(50) # 50 개만을 선택해서 뽑음

In [ ]:
plt.figure(figsize=(15,6)) #15, 6 크기의 그래프를 그리겠다
ko.plot(50) # 50개의 데이터만 사용
plt.show()

# 그래프에서 한글 폰트 깨지는 문제에 대한 대처(전역 글꼴 설정)
from matplotlib import font_manager, rc
font_name = font_manager.FontProperties(fname='c:/Windows/Fonts/malgun.ttf').get_name()
rc('font', family=font_name)

In [ ]:
data = ko.vocab().most_common(300) #가장 빈도수가 높은 300개 단어만 추출해서 data 변수에 저장

print(len(data))
data

In [ ]:
# list tuple을 딕셔너리로 만들어주는 함수
# 단어마다 빈도수를 찍어놓은 형태가 현재 리스트 안의 튜플 속에 있는 데 이를 딕셔너리로 만들려고 함

def todict(list_tuple):    
    todict = {}
    for i in range(0,len(list_tuple)):
        todict[data[i][0]] = data[i][1]
    return todict

In [ ]:
# 워드클라우드를 그려보자
wordcloud = WordCloud(font_path='c:/Windows/Fonts/malgun.ttf',
                      relative_scaling = 0.2,
                      #stopwords=STOPWORDS,
                      background_color='white',
                      ).generate_from_frequencies(todict(data))

plt.figure(figsize=(16,8))
plt.imshow(wordcloud)
plt.axis("off")
plt.show()